In [28]:
import sympy as symp
symp.init_printing("mathjax")
import numpy as np

print("SymPy Version",symp.__version__)
print("NumPy Version",np.__version__)

SymPy Version 1.12
NumPy Version 1.25.2


In [29]:
# Test right hand side multiplication with vector if ok
A = symp.Matrix([[1,2,3],[4,5,6],[7,8,9]])
b = symp.Matrix([1,2,3])
print(A*b)  # Matrix([[14], [32], [50]])  Ok

Matrix([[14], [32], [50]])


In [30]:
def print_and_simplify_vector(expr, name):
    print(f"{name}:")
    print("-----")
    for idx, expr in enumerate(expr):
      expr =  symp.nsimplify(expr)
      print(expr, end=", ")
    print(f"\n")

def print_and_simplify_mat(expr, name):
    print(name,":")
    print("-----")
    for idx1 in range(expr.shape[0]):
        for idx2 in range(expr.shape[1]):
            expr[idx1,idx2]= symp.nsimplify(expr[idx1,idx2])
            print(expr[idx1,idx2],end=" , ")
        print("\n")
    print("\n")    

In [31]:
def skew_symmetric(v):
    if len(v) != 3:
        raise ValueError("The vector should have 3 elements.")

    return symp.Matrix([
        [0, -v[2], v[1]],
        [v[2], 0, -v[0]],
        [-v[1], v[0], 0]
    ])

In [32]:
def Rx(theta):
  return symp.Matrix([[ 1.0, 0.0           , 0.0           ],
                   [ 0.0, symp.cos(theta), -symp.sin(theta)],
                   [ 0.0, symp.sin(theta), symp.cos(theta)]])

def Ry(theta):
  return symp.Matrix([[ symp.cos(theta), 0.0, symp.sin(theta)],
                   [ 0.0            , 1.0, 0.0           ],
                   [ -symp.sin(theta), 0.0, symp.cos(theta)]])

def Rz(theta):
  return symp.Matrix([[ symp.cos(theta), -symp.sin(theta), 0.0],
                   [ symp.sin(theta), symp.cos(theta) , 0.0 ],
                   [ 0.0 , 0.0   , 1.0 ]])

In [33]:
def compute_jacobian(F, X):
    """
    Compute the Jacobian of the vector F with respect to vector x and print it.

    Args:
    - F (Matrix): m x 1 symbolic matrix (vector) for which Jacobian is computed
    - x (Matrix): n x 1 symbolic matrix (vector) with respect to which Jacobian is computed

    Returns:
    - Matrix: m x n Jacobian matrix
    """
    # Compute the Jacobian matrix
    J = F.jacobian(X)
   
    # Print the result
    for idx1 in range(J.shape[0]):
        for idx2 in range(J.shape[1]):
          J[idx1,idx2]= symp.nsimplify(J[idx1,idx2])
    
    return J

In [34]:
# Equation set 1 - Euler ve
# X = [pos,euler,ve,bg,ba]
# U = [meas_ab, meas_ob]
# H = [pos, vb]

X = symp.symbols('x y z roll pitch yaw vex vey vez bgx bgy bgz bax bay baz')
print_and_simplify_vector(X, "X")

U = symp.symbols('abx aby abz ox1 ox2 ox3') 
print_and_simplify_vector(U, "U")

# util exprs
ve = symp.Matrix([X[6], X[7], X[8]])
ab = symp.Matrix([ U[0], U[1], U[2] ])
ob = symp.Matrix([ U[3], U[4], U[5] ])
cr = symp.cos(X[3])
sr = symp.sin(X[3])
cp = symp.cos(X[4])
sp = symp.sin(X[4])
Einv = 1.0/cp*symp.Matrix([ [ cp, sr*sp, cr*sp ], [ 0, cr*cp, -sr*cp ], [ 0, sr, cr ] ])
Reb = Rz(X[5])*Ry(X[4])*Rx(X[3])
Rbe = Rx(X[3]).T*Ry(X[4]).T*Rz(X[5]).T

# The dynamics equation dF(X)/dt
d_pos = ve
d_rpy = Einv*ob 
d_ve = Reb*ab + symp.Matrix([0,0,-9.81])
d_bg = symp.Matrix([0,0,0])
d_ba = symp.Matrix([0,0,0])
# stack
F = symp.Matrix.vstack(d_pos,d_rpy)
F = symp.Matrix.vstack(F,d_ve)
F = symp.Matrix.vstack(F,d_bg)
F = symp.Matrix.vstack(F,d_ba)
print_and_simplify_vector(F, "dF(X)dt")
print_and_simplify_vector(F, "dF(X)dt")

# The measurement equation H(X) 
H1 = symp.Matrix([X[0],X[1],X[2]])
H2 = symp.Matrix(Rbe*ve)
H = symp.Matrix.vstack(H1, H2)
print_and_simplify_vector(H,"H(X)")

### Calculate the Jacobian of F:
jacobian_F = compute_jacobian(F, X)
print_and_simplify_mat(jacobian_F, "dFdX")

### Calculate the Jacobian of H:
jacobian_H = compute_jacobian(H, X)
print_and_simplify_mat(jacobian_H,"dHdX")

X:
-----
x, y, z, roll, pitch, yaw, vex, vey, vez, bgx, bgy, bgz, bax, bay, baz, 

U:
-----
abx, aby, abz, ox1, ox2, ox3, 

dF(X)dt:
-----
vex, vey, vez, ox1 + ox2*sin(pitch)*sin(roll)/cos(pitch) + ox3*sin(pitch)*cos(roll)/cos(pitch), ox2*cos(roll) - ox3*sin(roll), ox2*sin(roll)/cos(pitch) + ox3*cos(roll)/cos(pitch), abx*cos(pitch)*cos(yaw) + aby*(sin(pitch)*sin(roll)*cos(yaw) - sin(yaw)*cos(roll)) + abz*(sin(pitch)*cos(roll)*cos(yaw) + sin(roll)*sin(yaw)), abx*sin(yaw)*cos(pitch) + aby*(sin(pitch)*sin(roll)*sin(yaw) + cos(roll)*cos(yaw)) + abz*(sin(pitch)*sin(yaw)*cos(roll) - sin(roll)*cos(yaw)), -abx*sin(pitch) + aby*sin(roll)*cos(pitch) + abz*cos(pitch)*cos(roll) - 981/100, 0, 0, 0, 0, 0, 0, 

dF(X)dt:
-----
vex, vey, vez, ox1 + ox2*sin(pitch)*sin(roll)/cos(pitch) + ox3*sin(pitch)*cos(roll)/cos(pitch), ox2*cos(roll) - ox3*sin(roll), ox2*sin(roll)/cos(pitch) + ox3*cos(roll)/cos(pitch), abx*cos(pitch)*cos(yaw) + aby*(sin(pitch)*sin(roll)*cos(yaw) - sin(yaw)*cos(roll)) + abz*(sin(pitch

In [37]:
# Equation set 2 - Euler vb
# X = [pos,euler,vb,bg,ba]
# U = [meas_ab, meas_ob]
# H = [pos, vb]

X = symp.symbols('x y z roll pitch yaw vbx vby vbz bgx bgy bgz bax bay baz')
print_and_simplify_vector(X, "X")

U = symp.symbols('abx aby abz ox oy oz') 
print_and_simplify_vector(U, "U")

# util exprs
vb = symp.Matrix([X[6], X[7], X[8]])
ab = symp.Matrix([ U[0], U[1], U[2] ])
ob = symp.Matrix([ U[3], U[4], U[5] ])
cr = symp.cos(X[3])
sr = symp.sin(X[3])
cp = symp.cos(X[4])
sp = symp.sin(X[4])
Einv = 1.0/cp*symp.Matrix([ [ cp, sr*sp, cr*sp ], [ 0, cr*cp, -sr*cp ], [ 0, sr, cr ] ])
Reb = Rz(X[5])*Ry(X[4])*Rx(X[3])
Rbe = Rx(X[3]).T*Ry(X[4]).T*Rz(X[5]).T

# The dynamics equation dF(X)/dt
d_pos = Reb*vb
d_rpy = Einv*ob
d_vb = -skew_symmetric(ob)*vb + ab

# stack
F = symp.Matrix.vstack(d_pos,d_rpy)
F = symp.Matrix.vstack(F,d_vb)
print_and_simplify_vector(F, "dF(X)dt")

# The measurement equation H(X)
H1 = symp.Matrix([X[0],X[1],X[2]])  # pos 
H2 = symp.Matrix(vb)            # odometry
H = symp.Matrix.vstack(H1, H2)
print_and_simplify_vector(H,"H(X)")

### Calculate the Jacobian:
jacobian_F = compute_jacobian(F, X)
print_and_simplify_mat(jacobian_F, "dFdX")

### Calculate the Jacobian
jacobian_H = compute_jacobian(H, X)
print_and_simplify_mat(jacobian_H,"dHdX")

X:
-----
x, y, z, roll, pitch, yaw, vbx, vby, vbz, bgx, bgy, bgz, bax, bay, baz, 

U:
-----
abx, aby, abz, ox1, ox2, ox3, 

dF(X)dt:
-----
vbx*cos(pitch)*cos(yaw) + vby*(sin(pitch)*sin(roll)*cos(yaw) - sin(yaw)*cos(roll)) + vbz*(sin(pitch)*cos(roll)*cos(yaw) + sin(roll)*sin(yaw)), vbx*sin(yaw)*cos(pitch) + vby*(sin(pitch)*sin(roll)*sin(yaw) + cos(roll)*cos(yaw)) + vbz*(sin(pitch)*sin(yaw)*cos(roll) - sin(roll)*cos(yaw)), -vbx*sin(pitch) + vby*sin(roll)*cos(pitch) + vbz*cos(pitch)*cos(roll), ox1 + ox2*sin(pitch)*sin(roll)/cos(pitch) + ox3*sin(pitch)*cos(roll)/cos(pitch), ox2*cos(roll) - ox3*sin(roll), ox2*sin(roll)/cos(pitch) + ox3*cos(roll)/cos(pitch), abx - ox2*vbz + ox3*vby, aby + ox1*vbz - ox3*vbx, abz - ox1*vby + ox2*vbx, 

H(X):
-----
x, y, z, vbx, vby, vbz, 

dFdX :
-----
0 , 0 , 0 , vby*(sin(pitch)*cos(roll)*cos(yaw) + sin(roll)*sin(yaw)) + vbz*(-sin(pitch)*sin(roll)*cos(yaw) + sin(yaw)*cos(roll)) , -vbx*sin(pitch)*cos(yaw) + vby*sin(roll)*cos(pitch)*cos(yaw) + vbz*cos(pitch)*co

In [36]:
# Equation set 3 - Euler vb ob ab 
# X = [pos, euler, vb, ob, ab, bg, ba]
# H = [pos, vb, ob+bg, ab+ba ]

X = symp.symbols('x y z roll pitch yaw vbx vby vbz ox oy oz ax ay az bgx bgy bgz bax bay baz')
print_and_simplify_vector(X, "X")

# util exprs
vb = symp.Matrix([ X[6], X[7], X[8] ])
ae = symp.Matrix([ X[12], X[13], X[14] ])
ob = symp.Matrix([ X[9], X[10], X[11] ])
ba = symp.Matrix([ X[18], X[19], X[20] ])
bg = symp.Matrix([ X[15], X[16], X[17] ])
cr = symp.cos(X[3])
sr = symp.sin(X[3])
cp = symp.cos(X[4])
sp = symp.sin(X[4])
Einv = 1.0/cp*symp.Matrix([ [ cp, sr*sp, cr*sp ], [ 0, cr*cp, -sr*cp ], [ 0, sr, cr ] ])
Reb = Rz(X[5])*Ry(X[4])*Rx(X[3])
Rbe = Rx(X[3]).T*Ry(X[4]).T*Rz(X[5]).T

# The dynamics equation dF(X)/dt
d_pos = Reb*vb
d_rpy = Einv*ob
d_vb = -skew_symmetric(ob)*vb + Rbe*ae
d_ob = symp.Matrix([0,0,0])
d_ae = symp.Matrix([0,0,0])
d_bg = symp.Matrix([0,0,0])
d_ba = symp.Matrix([0,0,0])

# stack
F = symp.Matrix.vstack(d_pos,d_rpy)
F = symp.Matrix.vstack(F,d_vb)
F = symp.Matrix.vstack(F,d_ob)
F = symp.Matrix.vstack(F,d_ae)
F = symp.Matrix.vstack(F,d_bg)
F = symp.Matrix.vstack(F,d_ba)
print_and_simplify_vector(F, "dF(X)dt")

# The measurement equation H(X)
H1 = symp.Matrix([X[0],X[1],X[2]])
H2 = symp.Matrix(vb)
H3 = symp.Matrix(ob+bg)
H4 = symp.Matrix(Rbe*(ae+symp.Matrix([0,0,-9.81]))+ba)
H = symp.Matrix.vstack(H1, H2)
H = symp.Matrix.vstack(H, H3)
H = symp.Matrix.vstack(H, H4)
print_and_simplify_vector(H,"H(X)")

### Calculate the Jacobian:
jacobian_F = compute_jacobian(F, X)
print_and_simplify_mat(jacobian_F, "dFdX")

### Calculate the Jacobian
jacobian_H = compute_jacobian(H, X)
print_and_simplify_mat(jacobian_H,"dHdX")

X:
-----
x, y, z, roll, pitch, yaw, vbx, vby, vbz, ox, oy, oz, ax, ay, az, bgx, bgy, bgz, bax, bay, baz, 

dF(X)dt:
-----
vbx*cos(pitch)*cos(yaw) + vby*(sin(pitch)*sin(roll)*cos(yaw) - sin(yaw)*cos(roll)) + vbz*(sin(pitch)*cos(roll)*cos(yaw) + sin(roll)*sin(yaw)), vbx*sin(yaw)*cos(pitch) + vby*(sin(pitch)*sin(roll)*sin(yaw) + cos(roll)*cos(yaw)) + vbz*(sin(pitch)*sin(yaw)*cos(roll) - sin(roll)*cos(yaw)), -vbx*sin(pitch) + vby*sin(roll)*cos(pitch) + vbz*cos(pitch)*cos(roll), ox + oy*sin(pitch)*sin(roll)/cos(pitch) + oz*sin(pitch)*cos(roll)/cos(pitch), oy*cos(roll) - oz*sin(roll), oy*sin(roll)/cos(pitch) + oz*cos(roll)/cos(pitch), ax*cos(pitch)*cos(yaw) + ay*sin(yaw)*cos(pitch) - az*sin(pitch) - oy*vbz + oz*vby, ax*(sin(pitch)*sin(roll)*cos(yaw) - sin(yaw)*cos(roll)) + ay*(sin(pitch)*sin(roll)*sin(yaw) + cos(roll)*cos(yaw)) + az*sin(roll)*cos(pitch) + ox*vbz - oz*vbx, ax*(sin(pitch)*cos(roll)*cos(yaw) + sin(roll)*sin(yaw)) + ay*(sin(pitch)*sin(yaw)*cos(roll) - sin(roll)*cos(yaw)) + az*co